In [ ]:
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.research import run_pipeline

from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import Q500US

from quantopian.pipeline.filters import StaticAssets

import pandas as pd

In [ ]:
def make_pipeline():
    pipe = Pipeline()
    return pipe 



In [ ]:
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline

from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import StaticAssets

import numpy 

class StdDev(CustomFactor):
    def compute(self, today, asset_ids, out, values): 
        out[:] = numpy.nanstd(values, axis=0)
        

def make_pipeline():
    close = USEquityPricing.close
    sma10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    stdev10 = StdDev(inputs=[close], window_length=10)
    my_symbols = StaticAssets(symbols(['STZ', 'MON', 'WBA', 'DAL', 'JPM', 'WFC', 'SPY']))

    
    pipe = Pipeline(columns = {'sma10': sma10,
                               'close': close.latest ,
                               'stdev10': stdev10
                              },
                   screen=my_symbols)
    return pipe 

results = run_pipeline(make_pipeline(), start_date='2017-1-5', end_date='2017-1-17')


In [ ]:
df = local_csv("nasdaq_earning_calendar.csv", date_column=":Time", use_date_column_as_index=True)
results.reset_index

In [ ]:
results['Symbol'] = [s.symbol for s in results.index.get_level_values(1)]
results['ExDate'] = pd.Series()
dates = results.index.get_level_values(0).unique()

for date in dates:
    print date
    s_earning = df.ix[date]['Symbol']
    
    
    if type(s_earning) ==  str:#pd.core.series.Series:
        s_earning = [s_earning]
        
    
    for s in s_earning:
        print s, results.ix[date]['Symbol'].tolist()
        if s in results.ix[date]['Symbol'].tolist():
            
            results.iloc[date]['ExDate'] = True
results

In [ ]:
s_earning = df.ix[dates[1]]['Symbol']
if type(s_earning) ==  str:#pd.core.series.Series:
    s_earning = [s_earning]

print(s_earning)
for s in s_earning:
    if s in results.ix[dates[1]]['Symbol']:
        results.iloc[dates[1]]['ExDate'] = True

    
results    
#df[df[":sp500?"]].sort_index().ix["2017-1-1":]


In [ ]:
import numpy as np
class SidInList(CustomFilter):
    """
    Filter returns True for any SID included in parameter tuple passed at creation.
    Usage: my_filter = SidInList(sid_list=(23911, 46631))
    """    
    inputs = []
    window_length = 1
    params = ('sid_list',)

    def compute(self, today, assets, out, sid_list):
        out[:] = np.in1d(assets, sid_list)

def make_pipeline():
    include_filter = SidInList(sid_list = (24, 8554)) # SID for APPL and SPY
    pipe = Pipeline(
        screen = include_filter
    )
    return pipe 
        
# include_filter = SidInList(sid_list = (24, 8554)) # SID for APPL and SPY

# pipe = Pipeline()
# pipe.set_screen(include_filter)

results = run_pipeline(pipe, '2016-07-01', '2016-07-01')
    
results
